In [45]:
%load_ext sql
import os
from sqlalchemy import create_engine

pgconfig = {
    'host': 'db',
    'port': os.environ['PG_PORT'],
    'database': os.environ['PG_DATABASE'],
    # see docker compose
    'user': 'postgres',
    'password': 'postgres12345',
}
# データベースはあらかじめ作っておく
dsl = 'postgresql://{user}:{password}@{host}:{port}/{database}'.format(**pgconfig)
conn = create_engine(dsl)
print(dsl)
# MagicコマンドでSQLを書くための設定
%sql conn

The sql extension is already loaded. To reload it, use:
  %reload_ext sql
postgresql://postgres:postgres12345@db:5432/dsdojo_db


```bash
> \d+
                                   リレーション一覧
 スキーマ |   名前   |  タイプ  | 所有者  | 永続性 | アクセスメソッド | サイズ  | 説明
----------+----------+----------+---------+--------+------------------+---------+------
 public   | category | テーブル | padawan | 永続   | heap             | 48 kB   |
 public   | customer | テーブル | padawan | 永続   | heap             | 3488 kB |
 public   | geocode  | テーブル | padawan | 永続   | heap             | 17 MB   |
 public   | product  | テーブル | padawan | 永続   | heap             | 704 kB  |
 public   | receipt  | テーブル | padawan | 永続   | heap             | 8672 kB |
 public   | store    | テーブル | padawan | 永続   | heap             | 48 kB   |
(6 行)

# database一覧
> \l
                                                           データベース一覧
   名前    |  所有者  | エンコーディング |  照合順序  | Ctype(変換演算子) | ICUロケール | ロケールプロバイダー |     アクセス権限
-----------+----------+------------------+------------+-------------------+-------------+----------------------+-----------------------
 dsdojo_db | padawan  | UTF8             | ja_JP.utf8 | ja_JP.utf8        |             | libc                 | =Tc/padawan          +
           |          |                  |            |                   |             |                      | padawan=CTc/padawan
 postgres  | postgres | UTF8             | ja_JP.utf8 | ja_JP.utf8        |             | libc                 |
 template0 | postgres | UTF8             | ja_JP.utf8 | ja_JP.utf8        |             | libc                 | =c/postgres          +
           |          |                  |            |                   |             |                      | postgres=CTc/postgres
 template1 | postgres | UTF8             | ja_JP.utf8 | ja_JP.utf8        |             | libc                 | =c/postgres          +
           |          |                  |            |                   |             |                      | postgres=CTc/postgres
```

In [49]:
%%sql
drop database mydb;
create database mydb;

   postgresql://padawan:***@db:5432/dsdojo_db
*  postgresql://postgres:***@db:5432/dsdojo_db
Done.
Done.


[]

In [53]:
%%sql
-- これでもOK
DROP ROLE testuser;
-- CREATE ROLE testuser LOGIN PASSWORD 'test';
CREATE USER testuser PASSWORD 'test';

   postgresql://padawan:***@db:5432/dsdojo_db
*  postgresql://postgres:***@db:5432/dsdojo_db
Done.
Done.


[]

In [55]:
%%sql
-- allの他にselect, insertなどを指定できる
-- -- database全体の場合は
grant all on database mydb to testuser;
-- GRANT ALL ON DATABASE mydb TO testuser; でも同じ
-- -- [WITH GRANT OPTION] を付けた場合、権限を与えられたロールは同じ権限を別のロールに追加出来るようになります
-- mydbの所有者をpostgres -> testuserに移動
-- -> testuserの権限でselectやinsertを行える
ALTER DATABASE mydb OWNER TO testuser;

   postgresql://padawan:***@db:5432/dsdojo_db
*  postgresql://postgres:***@db:5432/dsdojo_db
Done.
Done.


[]

In [58]:
# Python
# ユーザー変更
dsl = 'postgresql://testuser:test@{host}:{port}/mydb'.format(**pgconfig)
conn = create_engine(dsl)
print(dsl)
# MagicコマンドでSQLを書くための設定
%sql conn

postgresql://testuser:test@db:5432/mydb


In [59]:
%%sql
-- 権限付与したのでSQLクエリを実行できる
CREATE TABLE sample (id int, name varchar);
insert into sample values (1,'aaa');
select * from sample;

   postgresql://padawan:***@db:5432/dsdojo_db
   postgresql://postgres:***@db:5432/dsdojo_db
   postgresql://testuser:***@db:5432/dsdojo_db
*  postgresql://testuser:***@db:5432/mydb
Done.
1 rows affected.
1 rows affected.


id,name
1,aaa
